##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 케라스 모델로 추정기 생성하기

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/keras_model_to_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub) 소스 보기</a>
  </td>
</table>

Note: 이 문서는 텐서플로 커뮤니티에서 번역했습니다. 커뮤니티 번역 활동의 특성상 정확한 번역과 최신 내용을 반영하기 위해 노력함에도
불구하고 [공식 영문 문서](https://www.tensorflow.org/?hl=en)의 내용과 일치하지 않을 수 있습니다.
이 번역에 개선할 부분이 있다면
[tensorflow/docs](https://github.com/tensorflow/docs) 깃헙 저장소로 풀 리퀘스트를 보내주시기 바랍니다.
문서 번역이나 리뷰에 참여하려면
[docs-ko@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ko)로
메일을 보내주시기 바랍니다.

## 개요

텐서플로우는 [`tf.keras`](https://www.tensorflow.org/api_docs/python/tf/keras) 모델을 텐서플로우 추정기(estimator)로 손쉽게 변환하는 기능을 지원하고 있습니다. 본 튜토리얼에서는 이 변환 과정의 전반적인 예시를 소개합니다.

## 설정

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  # %tensorflow_version 명령은 코랩에서만 쓸 수 있습니다.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import tensorflow as tf

import numpy as np
import tensorflow_datasets as tfds

### 간단한 케라스 모델 만들기

케라스에서는 여러 겹의 층을 쌓아 모델을 만들 수 있습니다. 일반적으로 모델은 층의 그래프로 구성됩니다. 이 중 가장 흔한 형태는 적층형 구조를 갖고 있는 [`tf.keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) 모델입니다.

간단한 완전히 연결 네트워크(다층 퍼셉트론)를 만들어봅시다:

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3)
])

모델을 컴파일한 후, 모델 구조의 요약도를 볼 수 있습니다.

In [0]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam')
model.summary()

### 입력 함수 만들기

[데이터셋 API](../../guide/data.md)를 사용해 대규모 데이터셋을 확장하거나 다중 기기 학습(multi-device training)을 실행할 수 있습니다.

텐서플로우 추정기는 인풋 파이프라인(input pipeline)이 언제 어떻게 생성었는지 제어할 수 있어야 합니다. 이를 위해서는 "입력 함수", 즉 `input_fn`이 필요합니다. 추정기는 이 함수를 별도의 매개변수 설정 없이 호출하게 됩니다. 이때 `input_fn`은 `tf.data.Dataset` 객체를 반환해야 합니다.

In [0]:
def input_fn():
  split = tfds.Split.TRAIN
  dataset = tfds.load('iris', split=split, as_supervised=True)
  dataset = dataset.map(lambda features, labels: ({'dense_input':features}, labels))
  dataset = dataset.batch(32).repeat()
  return dataset

`input_fn`이 잘 구현되었는지 확인해봅니다.

In [0]:
for features_batch, labels_batch in input_fn().take(1):
  print(features_batch)
  print(labels_batch)

### tf.keras.model을 추정기로 변환하기

`tf.keras.model`은 `tf.keras.estimator.model_to_estimator` 함수를 이용해 케라스 모델을 `tf.estimator.Estimator` 객체로 변환함으로써 `tf.estimator` API를 이용해 학습시킬 수 있습니다.

In [0]:
import tempfile
model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model, model_dir=model_dir)

추정기를 학습시킨 후 평가합니다.

In [0]:
keras_estimator.train(input_fn=input_fn, steps=500)
eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10)
print('Eval result: {}'.format(eval_result))